In [2]:
import pandas as pd
import requests
from pandas.io.json import json_normalize
import warnings
import datetime
import geocoder
warnings.filterwarnings('ignore')


In [3]:
url = 'http://overpass-api.de/api/interpreter'  # Overpass API URL


In [41]:
cities_list = ['Yerevan, Armenia','Vienna, Austria','Baku, Azerbaijan','Ganja, Azerbaijan','Sumgayit, Azerbaijan','Minsk, Belarus','Dubrovnik, Croatia','Pula, Croatia','Rijeka, Croatia','Šibenik, Croatia','Split, Croatia','Zadar, Croatia','Zagreb, Croatia','Limassol, Cyprus','Nicosia, Cyprus','Prague, Czech Republic','Haapsalu, Estonia','Jõhvi, Estonia','Kohtla-Järve, Estonia','Kuressaare, Estonia','Narva, Estonia','Pärnu, Estonia','Rakvere, Estonia','Tallinn, Estonia','Tartu, Estonia','Viljandi, Estonia','Lyon, France','Paris, France','Batumi, Georgia','Kutaisi, Georgia','Tbilisi, Georgia','Accra, Ghana','Cape Coast, Ghana','Kumasi, Ghana','Takoradi, Ghana','Budapest, Hungary','Debrecen, Hungary','Pécs, Hungary','Baghdad, Iraq','Nur-Sultan, Kazakhstan','Diani, Kenya','Eldoret, Kenya','Embu, Kenya','Kakamega, Kenya','Karatina, Kenya','Kilifi, Kenya','Kisumu, Kenya','Kitale, Kenya','Malindi, Kenya','Meru, Kenya','Mombasa, Kenya','Nairobi, Kenya','Naivasha, Kenya','Nakuru, Kenya','Nanyuki, Kenya','Nyeri, Kenya','Thika, Kenya','Daugavpils, Latvia','Jelgava, Latvia','Liepāja, Latvia','Rīga, Latvia','Valmiera, Latvia','Ventspils, Latvia','Alytus, Lithuania','Kaunas, Lithuania','Klaipėda, Lithuania','Mažeikiai, Lithuania','Panevėžys, Lithuania','Šiauliai, Lithuania','Vilnius, Lithuania','Malta, Malta','Aguascalientes, Mexico','Cuernavaca, Mexico','Guadalajara, Mexico','Mexico City, Mexico','Monterrey, Mexico','Abeokuta, Nigeria','Abuja, Nigeria','Benin City, Nigeria','Calabar, Nigeria','Enugu, Nigeria','Ibadan, Nigeria','Kaduna, Nigeria','Kano, Nigeria','Lagos, Nigeria','Owerri, Nigeria','Port Harcourt, Nigeria','Uyo, Nigeria','Zaria, Nigeria','Białystok, Poland','Bielsko-Biała, Poland','Bydgoszcz, Poland','Częstochowa, Poland','Elbląg, Poland','Gorzów Wielkopolski, Poland','Kalisz, Poland','Kielce, Poland','Koszalin, Poland','Kraków, Poland','Legnica, Poland','Łódź, Poland','Lublin, Poland','Olsztyn, Poland','Opole, Poland','Płock, Poland','Poznań, Poland','Radom, Poland','Rzeszów, Poland','Silesia, Poland','Szczecin, Poland','Tarnów, Poland','Toruń, Poland','Trójmiasto, Poland','Wałbrzych, Poland','Warsaw, Poland','Włocławek, Poland','Wrocław, Poland','Zielona Góra, Poland','Algarve, Portugal','Aveiro, Portugal','Barcelos, Portugal','Braga, Portugal','Coimbra, Portugal','Evora, Portugal','Leiria, Portugal','Lisbon, Portugal','Madeira, Portugal','Porto, Portugal','Viseu, Portugal','Bucharest, Romania','Cluj-Napoca, Romania','Timisoara, Romania','Saint Petersburg, Russia','Jazan, Saudi Arabia','Riyadh, Saudi Arabia','Belgrade, Serbia','Banska Bystrica, Slovakia','Bratislava, Slovakia','Kosice, Slovakia','Nitra, Slovakia','Prešov, Slovakia','Trnava, Slovakia','Žilina, Slovakia','Cape Town, South Africa','Durban, South Africa','East London, South Africa','Emalahleni, South Africa','Ermelo, South Africa','Garden Route, South Africa','Grahamstown, South Africa','Johannesburg, South Africa','Kimberley, South Africa','Mbombela, South Africa','Mthatha, South Africa','Phuthaditjhaba, South Africa','Pietermaritzburg, South Africa','Polokwane, South Africa','Port Elizabeth, South Africa','Potchefstroom, South Africa','Pretoria, South Africa','Queenstown, South Africa','Rustenburg, South Africa','Thohoyandou, South Africa','Upington, South Africa','Welkom, South Africa','Madrid, Spain','Gothenburg, Sweden','Stockholm, Sweden','Tunis, Tunisia','Kampala, Uganda','Kharkiv, Ukraine','Kyiv, Ukraine','Lviv, Ukraine','Odesa, Ukraine','London, United Kingdom','Arusha, United Republic of Tanzania','Dar es Salaam, United Republic of Tanzania','Dodoma, United Republic of Tanzania','Mwanza, United Republic of Tanzania']

In [40]:
city_dates_list = list()
for city in cities_list:
    try:
        g = geocoder.osm(city)
        osm_type = g.json['osm_type']
        if osm_type == 'relation':
            osm_search_id = 3600000000 + int((g.json['osm_id']))
        else:
            osm_search_id = 2400000000 + int((g.json['osm_id']))
        query = """[out:json];area({})->.searchArea;(way["tourism"="hotel"](newer:"2015-01-01T00:00:00Z")(area.searchArea);way["aeroway"="aerodrome"](newer:"2015-01-01T00:00:00Z")(area.searchArea););out meta;""".format(osm_search_id)
        query_results = requests.get(url, params={'data': query})
        query_data = query_results.json()['elements']
        query_df = json_normalize(query_data)
        query_df_clipped = query_df[['id','tags.name','timestamp']]
        query_df_clipped['date'] = pd.to_datetime(query_df_clipped.apply(lambda row: row['timestamp'].split('T')[0], axis = 1), infer_datetime_format=True)
        query_df_clipped = query_df_clipped.drop('timestamp', axis = 1)
        avg_edit_age = datetime.datetime.strftime(datetime.datetime.fromtimestamp(sum(map(datetime.datetime.timestamp,query_df_clipped['date']))/len(query_df_clipped['date'])),"%Y-%m-%d")
        city_dates_list.append((city, avg_edit_age))
    except:
        pass

{'accuracy': 0.7470254946139911, 'address': 'Երևան, Armenia', 'bbox': {'northeast': [40.26241, 44.6460194], 'southwest': [40.0656851, 44.4115546]}, 'city': 'Երևան', 'confidence': 1, 'country': 'Armenia', 'country_code': 'am', 'icon': 'https://nominatim.openstreetmap.org/images/mapicons/poi_boundary_administrative.p.20.png', 'importance': 0.7470254946139911, 'lat': 40.1776121, 'lng': 44.5125849, 'ok': True, 'osm_id': 364087, 'osm_type': 'relation', 'place_id': 235459230, 'place_rank': 7, 'quality': 'administrative', 'raw': {'place_id': 235459230, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 364087, 'boundingbox': ['40.0656851', '40.26241', '44.4115546', '44.6460194'], 'lat': '40.1776121', 'lon': '44.5125849', 'display_name': 'Երևան, Armenia', 'place_rank': 7, 'category': 'boundary', 'type': 'administrative', 'importance': 0.7470254946139911, 'icon': 'https://nominatim.openstreetmap.org/images/mapicons/poi_boundary

In [30]:
city_Dates_df = pd.DataFrame(city_dates_list,columns=['city', 'avgEditDate'])

In [38]:
oldest = city_Dates_df.sort_values(['avgEditDate']).head(1)
oldest

,city,avgEditDate
88,"Banska Bystrica, Slovakia",2016-06-08
